# **Airbnb**

Airbnb es un mercado en línea y servicio de hospitalidad, que permite a las personas arrendar o alquilar alojamiento a corto plazo. Para que Airbnb brinde una experiencia personalizada a sus clientes, busca explorar la posibilidad de predecir el país de destino en el que un usuario realizará una reserva. Con esta información, Airbnb puede crear contenido más personalizado con su comunidad, disminuir el tiempo promedio para la primera reserva y mejorar la previsión de la demanda. Estos objetivos proporcionan beneficios mutuos para Airbnb y sus clientes: las recomendaciones personales pueden mejorar el compromiso de los clientes con la plataforma, lo que fomenta las reservas repetidas y las referencias a Airbnb para aquellos los amigos cercanos y familiares. En este ejercicio tenemos los datos de los clientes de Airbnb y se busca predecir el primer destino de reserva para los nuevos clientes de Airbnb que viajan desde los Estados Unidos. La variable de respuesta es el destino donde se realiza la reserva.

# Predicción de destinos de Airbnb


Este notebook demuestra todo el proceso de construcción de un modelo predictivo para sugerir el primer destino de los nuevos usuarios de Airbnb. Todos los procesos involucrados, tales como la ordenación de datos, el análisis exploratorio de datos, las estadísticas inferenciales y el aprendizaje automático, se han dividido en secciones principales.

**CONJUNTO DE DATOS**

Para este ejercicio se contara con el siguiente conjuntos de datos:

Una lista de usuarios junto con sus datos demográficos. Registros de sesiones web y algunas estadísticas de resumen. Debemos predecir en qué país será el primer destino de la reserva de un nuevo usuario. Todos los usuarios en el conjunto de datos son de los Estados Unidos.

Hay 12 resultados posibles del país de destino: ' US ',' FR ',' CA ',' GB ',' ES ',' IT ',' PT ',' NL ',' DE ',' AU ', 'NDF' (no se encontró destino), y 'otro'. 'otro' significa que hubo una reserva, pero es para un país que no está incluido en la lista, mientras que 'NDF' significa que no había una reserva.

Los conjuntos de entrenamiento y prueba están divididos por fechas. El conjunto de pruebas contiene nuevos usuarios con primeras actividades después del 07/01/2014. En el conjunto de datos de las sesiones, los datos solo se remontan al 01/01/2014, mientras que el conjunto de datos de los usuarios se remonta a 2010.

## Preprocesamiento de Datos
En este paso vamos a limpiar los datos. 
Todo el Preprocesamiento de datos se realizará utilizando la biblioteca de Python Pandas.


In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

El primer paso es cargar todos los datos disponibles para nosotros en un Pandas Dataframe y extraer información básica como el número de muestras, el número de filas de valores nulos, el número de funciones, etc. 
El siguiente paso sería tratar los valores faltantes usando un método adecuado (eliminación, interpolación, etc.) y convertir ciertas características en una forma más adecuada para aplicar estadísticas inferenciales y machine learning algorithms.

### Estadística de edad, género y población


In [ ]:
df_agb = pd.read_csv('../input/age_gender_bkts.csv')
df_agb.head()

In [ ]:
df_agb.isnull().values.any()

Este Dataset no tiene valores nulos entre sus características.

Como primer paso, tratemos el dato de la edad. Las edades se indican en bloques de tamaño 5. Convertiremos esto en un valor numérico para que podamos aumentar su versatilidad si fuera necesario (es decir, se puede usar tanto como una característica categórica como continua). Como el tamaño del bloque es razonablemente pequeño, podemos definir una nueva función, **mean_age** que puede representar edades sin demasiada pérdida de información. El campo **mean_age** será el promedio del rango superior e inferior para ese bloque en particular.

In [ ]:
#Convert 100+ into a bin.
df_agb['age_bucket'] = df_agb['age_bucket'].apply(lambda x: '100-104' if x == '100+' else x)
#Define mean_age feature
df_agb['mean_age'] = df_agb['age_bucket'].apply(lambda x: (int(x.split('-')[0]) + int(x.split('-')[1]))/2)
df_agb = df_agb.drop('age_bucket', axis=1)
df_agb.head()

In [ ]:
df_agb['country_destination'].value_counts()

El número de países es extremadamente pequeño. Probablemente corresponda a las clases por país disponibles en el conjunto de datos de capacitación que cubriremos más adelante en esta sección.

In [ ]:
df_agb['gender'].value_counts()

El género también se puede convertir en una variable binaria categórica. Representemos ** hombres con 0 y mujeres con 1. ** De nuevo, hacemos esto en caso de que necesitemos que esta variable funcione como una cantidad numérica. Debe indicarse que no hay una necesidad inmediata de ello y, por lo tanto, se puede omitir.

In [ ]:
df_agb['gender'] = df_agb['gender'].apply(lambda x: 0 if x == 'male' else 1)
df_agb['gender'].value_counts()

In [ ]:
df_agb['year'].value_counts()

Solo hay un valor disponible para la característica del año. Por lo tanto, no tiene sentido mantener esta función, ya que no agrega ninguna información sobre los datos disponibles para nosotros. Dejaremos caer esta característica.

In [ ]:
df_agb = df_agb.drop('year', axis=1)
df_agb.head()

### Estadisticas de País

In [ ]:
df_con = pd.read_csv('../input/countries.csv')
df_con

Como tal, no necesitamos realizar ninguna limpieza para este conjunto de datos. Las estadísticas que se proporcionan aquí deberán incorporarse a nuestros conjuntos de datos de capacitación y pruebas antes de capacitar a nuestro modelo y/o se tendrán que utilizar en el análisis de datos exploratorios para obtener información adicional.

### Sesiones


In [ ]:
df_ses = pd.read_csv('../input/sessions.csv')
df_ses.head(15)

In [ ]:
df_ses.shape

Reemplazaremos los valores desconocidos con NaN para darle algún significado semántico esto dara lugar a mejores predicciones.

In [ ]:
df_ses['action'] = df_ses['action'].replace('-unknown-', np.nan)
df_ses['action_type'] = df_ses['action_type'].replace('-unknown-', np.nan)
df_ses['action_detail'] = df_ses['action_detail'].replace('-unknown-', np.nan)

Primero, revisemos la variable secs_elapsed.

In [ ]:
sns.distplot(df_ses[df_ses['secs_elapsed'].notnull()]['secs_elapsed'])

In [ ]:
df_ses['secs_elapsed'].describe()

El número promedio de segundos por sesión es ** 19450 segundos ** mientras que el valor de la mediana es ** 1147 segundos **. Esto sugiere un conjunto de datos muy sesgado con algunas observaciones atípicas que influyen en la media de la distribución.

In [ ]:
len(df_ses[df_ses['secs_elapsed'].isnull()])

Hay más de 136,000 campos para los cuales no sabemos el valor. Necesitamos descartar o interpolar estos valores. Por el momento, llenemos los valores no disponibles con la mediana de la distribución. Utilizamos la mediana ya que, como se mencionó anteriormente, esta distribución está fuertemente influenciada por valores atípicos.

In [ ]:
median_secs = df_ses['secs_elapsed'].median()
df_ses['secs_elapsed'] = df_ses['secs_elapsed'].fillna(median_secs)

In [ ]:
df_ses['secs_elapsed'].describe()

Como se puede ver anteriormente, la interpolación no ha alterado significativamente las estadísticas de resumen de la distribución. Esta es una buena opción para completar los valores nulos por el momento.

In [ ]:
null_action = df_ses[(df_ses['action_type'].isnull()) | (df_ses['action_detail'].isnull()) | (df_ses['action'].isnull()) ]
null_action.head()

In [ ]:
null_action.shape

In [ ]:
len(null_action['action'].drop_duplicates())

Hay más de 2,23 millones de campos que tienen información incompleta sobre la acción, el tipo de acción o el detalle de la acción. Esto es aproximadamente el 20% de los datos que poseemos. Además, hay 270 tipos de acciones para las cuales tenemos valores nulos, por lo que examinarlos uno por uno simplemente no es posible de forma manual.

Dado que este no es nuestro conjunto de datos de entrenamiento, no descartaremos los valores. Lo mantendremos porque es posible extraer información incluso de filas que tienen columnas nulas.

In [ ]:
df_ses[df_ses['device_type'].isnull()]

In [ ]:
df_ses['device_type'] = df_ses['device_type'].replace('-unknown-', np.nan)

In [ ]:
df_ses['device_type'].value_counts()

Como se puede ver arriba, hay 13 tipos de dispositivos que las personas están usando. También hay cierta actividad para la cual no tenemos datos sobre el dispositivo. Veamos cuántas instancias tiene esa cuenta.

In [ ]:
len(df_ses[df_ses['device_type'].isnull()])

Al igual que con las variables de acción, no descartaremos estos valores, ya que este no es el conjunto de datos al que aplicaremos nuestros algoritmos de aprendizaje automático. Sin embargo, reemplazaremos las cadenas desconocidas con NaN para darle un significado más semántico que podría ser útil en nuestro análisis.

### Usuarios de Entrenamiento

Ahora exploremos el conjunto de datos de entrenamiento que consiste en nuestros usuarios para quienes necesitamos predecir el primer destino.

In [ ]:
df_train = pd.read_csv('../input/train_users_2.csv')
df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train['gender'] = df_train['gender'].replace('-unknown-', np.nan)
df_train['first_browser'] = df_train['first_browser'].replace('-unknown-', np.nan)

In [ ]:
df_train[df_train['first_device_type'].isnull()]

Las siguientes características en el conjunto de datos no tienen valores nulos:

1. date_account_created
2. timestamp_first_active
3. signup_method
4. signup_flow
5. language
6. affiliate_channel
7. affiliate_provider
8. signup_app
9. first_device_type

Las siguientes características en el conjunto de datos tienen valores nulos:

1. date_first_booking
2. gender
3. age
4. first_affiliate_tracked
5. first_browser

In [ ]:
df_train[df_train['age'] > 120].head()

Notamos que hay muestras para las cuales la edad es mucho mayor que 120 (la persona mayor tiene actualmente 114 años). Se trata de datos claramente contaminados y deben ser eliminados. Reemplazemos estos valores con NaN para indicar que no conocemos la edad real de estas personas.

In [ ]:
df_train['age'] = df_train['age'].apply(lambda x: np.nan if x > 120 else x)

## Estadística inferencial

### Preferencia de género para las reservaciones de Airbnb


En esta sección, aplicaremos herramientas estadísticas para obtener algunas inferencias e ideas sobre el tipo de datos con los que estamos tratando y cómo eliminar las relaciones entre varias características de nuestro conjunto de datos.

Para comenzar, verifiquemos si hay una preferencia basada en el género para ciertos países. En otras palabras, ¿el género de una persona afecta al primer país en el que reserva un Airbnb? Para responder a esta pregunta tendremos que probar la relación entre dos variables categóricas: Género y País de destino. Dado que el número de países de destino es multivariado, la ** Prueba de Significación  de Chi Cuadrada**.

Antes de comenzar, haremos ciertas suposiciones:
* Consideraremos solo a aquellos usuarios que hayan reclutado su género como hombres o mujeres. Desconocidos y otros géneros no se incluyen en este análisis.

* No consideramos a los usuarios que nunca han reservado un Airbnb o que han reservado en un país que no está inscrito como clase (NDF y Otro).

In [ ]:
df_inf = df_train[(df_train['country_destination'] != 'NDF') & (df_train['country_destination'] != 'other') & (df_train['gender'] != 'OTHER') & (df_train['gender'].notnull())]
df_inf = df_inf[['id', 'gender', 'country_destination']]
df_inf.head()

In [ ]:
df_inf['gender'].value_counts()

In [ ]:
df_inf['country_destination'].value_counts()

#### Prueba de hipótesis

Para la prueba de hipótesis, definimos lo siguiente:

* ** Hipótesis nula: ** No existe una relación entre la preferencia del país y el sexo del cliente.
* ** Hipótesis alternativa: ** Existe una relación entre la preferencia del país y el sexo del cliente.

Asumiremos que nuestro nivel de significación, $ \ alpha $ sea 0.05.

In [ ]:
observed = df_inf.pivot_table('id', ['gender'], 'country_destination', aggfunc='count').reset_index()
del observed.columns.name
observed = observed.set_index('gender')
observed

In [ ]:
chi2, p, dof, expected = stats.chi2_contingency(observed)

In [ ]:
chi2

In [ ]:
p

El p-value que hemos obtenido es menor que nuestro nivel de significación elegido. Por lo tanto, rechazamos la hipótesis nula y aceptamos la hipótesis de alteración negativa. ** Existe una relación entre la preferencia de país y el sexo del cliente. ** Esta información puede ser extremadamente útil para nosotros cuando realizamos ingeniería de características en pasos posteriores.


### Preferencias de registro


En esta sección, trataremos de descubrir una relación entre el método de registro y el dispositivo de registro. Una consulta preliminar de nuestro marco de datos nos dice que la mayoría de los usuarios se registran a través del método básico estándar o de Facebook. Además, inician sesión utilizando un navegador de escritorio, un navegador móvil o una aplicación móvil.

¿El tipo de dispositivo (móvil o computadora) afecta el método de registro en Airbnb? Intentaremos responder a esta pregunta aquí. Una vez más, como antes, haremos ciertas suposiciones.
* Ignoraremos el método de registro de Google ya que hay muy pocas instancias de él.
* Consideraremos que iOs, Moweb y Android son del tipo de dispositivo móvil.

In [ ]:
df_signup = df_train[(df_train['signup_method'] != 'google')][['id', 'signup_method', 'signup_app']]
df_signup['device'] = df_signup['signup_app'].apply(lambda x: 'Computer' if x == 'Web' else 'Mobile')
df_signup.head()

In [ ]:
df_signup['signup_method'].value_counts()

In [ ]:
df_signup['device'].value_counts()

#### Prueba de hipótesis

Vamos a configurar ahora nuestra prueba de hipótesis. Definiremos lo siguiente:

* ** Hipótesis nula: ** No existe una relación entre el tipo de dispositivo y el método de registro.
* ** Hipótesis alternativa: ** Existe una relación entre el tipo de dispositivo y el método de registro.

Suponga que el nivel de significación $ \ alpha $ está en 0.05.

Realizaremos la misma prueba de hipótesis utilizando dos herramientas estadísticas diferentes: la prueba de significación de Chi cuadrado y la prueba de estadística Z. Si se realiza correctamente, ambas pruebas deben darnos los mismos resultados.

In [ ]:
df_signup = df_signup.pivot_table('id', ['device'], 'signup_method', aggfunc='count')
df_signup.index = ['Computer', 'Mobile']
df_signup.columns = ['Basic', 'Facebook']
df_signup

#### Prueba Chi Cuadrado

In [ ]:
chi2, p, dof, expected = stats.chi2_contingency(df_signup, correction=False)

In [ ]:
chi2

In [ ]:
p

El p-value obtenido es más alto que nuestro nivel de significación elegido. Por lo tanto, no estamos en posición de rechazar la hipótesis nula. Se destaca. ** No existe una relación entre el tipo de dispositivo y el método de registro. ** Realicemos la misma prueba de hipótesis utilizando la Prueba Z y comparemos los resultados y el p-value obtenido.

#### Prueba de significancia de dos muestras

Ahora hagamos la misma prueba de hipótesis utilizando la prueba de significación de dos muestras para proporciones.

In [ ]:
df_signup.loc['Total'] = [ df_signup['Basic'].sum(), df_signup['Facebook'].sum()]
df_signup['Total'] = df_signup['Basic'] + df_signup['Facebook']
df_signup

In [ ]:
fb_prop = df_signup.loc['Mobile', 'Facebook']/df_signup.loc['Total', 'Facebook']
fb_std = df_signup.loc['Mobile', 'Facebook'] * ((1 - fb_prop) ** 2) + df_signup.loc['Computer', 'Facebook'] * ((0 - fb_prop) ** 2)
fb_std = np.sqrt(fb_std/df_signup.loc['Total', 'Facebook'])

fb_prop, fb_std

In [ ]:
basic_prop = df_signup.loc['Mobile', 'Basic']/df_signup.loc['Total', 'Basic']
basic_std = df_signup.loc['Mobile', 'Basic'] * ((1 - basic_prop) ** 2) + df_signup.loc['Computer', 'Basic'] * ((0 - basic_prop) ** 2)
basic_std = np.sqrt(basic_std/df_signup.loc['Total', 'Basic'])

basic_prop, basic_std

In [ ]:
h0_prop = 0

prop_diff = fb_prop - basic_prop
p_hat = (df_signup.loc['Mobile', 'Basic'] + df_signup.loc['Mobile', 'Facebook'])/(df_signup.loc['Total', 'Basic'] + df_signup.loc['Total', 'Facebook']) 
var_diff = p_hat * (1- p_hat) * (1/df_signup.loc['Total', 'Basic'] + 1/df_signup.loc['Total', 'Facebook'])
sigma_diff = np.sqrt(var_diff)

prop_diff, sigma_diff

In [ ]:
z = (prop_diff - h0_prop) / sigma_diff
z

In [ ]:
p = (1-stats.norm.cdf(z))*2
p

El p-value obtenido es más alto que nuestro nivel de significancia elegido. Por lo tanto, no estamos en posición de rechazar la hipótesis nula. Se destaca. ** No existe una relación entre el tipo de dispositivo y el método de registro. **

También tenga en cuenta que los p-value obtenidos para la Prueba de Significación de Dos Muestras y la Prueba de Significación de Chi Cuadrado (sin corrección) son ** exactamente iguales. ** Ahora estamos en condiciones de resumir y presentar nuestros resultados en función del análisis estadístico realizado.

### Conclusiones

* Hay ** una relación ** entre género y Airbnb Country Destination. En otras palabras, el género influye en el país de su elección.
* No hay ** relación ** entre el tipo de dispositivo y el método de registro. Las dos variables son independientes entre sí.
* Los resultados obtenidos de la Prueba de Significación de Chi Cuadrado y la Prueba de Significación de Dos Muestras son ** exactamente los mismos **.

## Análisis exploratorio de datos

En esta sección, intentaremos visualizar y resumir nuestros datos para extraer nuevos conocimientos sobre las características individuales y la relación que comparten con las otras características. Primero intentemos y visualicemos las estadísticas de edad y género de varios países.



### Estadísticas de País

In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(x='mean_age', y='population_in_thousands', hue='gender', data=df_agb, ci=None)

**#### Observaciones

* Los países que están representados en esta estadística consisten en gran parte de un ** envejecimiento de la población **. Los grupos más grandes son personas con ** edad media de 47 años y 52 años **.
* La distribución se asemeja a una curva de campana sesgada. Las personas de mediana edad ocupan la mayor parte de la población, seguidas de cerca por los jóvenes y, finalmente, por los ancianos.
* Los conteos poblacionales de personas jóvenes y de mediana edad son bastante comparables. Pero a medida que hacemos la transición hacia la vejez (edad> 57 años), el conteo de la población para cada grupo sucesivo disminuye constantemente.
* Una cosa muy interesante a tener en cuenta (aunque puede no ser relevante para nuestro análisis) es que la proporción de sexos se inclina hacia los hombres para los grupos de edad más jóvenes, pero a medida que aumenta la edad media, la proporción se inclina más hacia las mujeres. ** Las mujeres de hecho viven más que los hombres. **

A continuación, tratemos de graficar el conteo de la población en cada país.

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,7))
pop_stats = df_agb.groupby('country_destination')['population_in_thousands'].sum()
sns.barplot(x=pop_stats.index, y=pop_stats)

** Los Estados Unidos de América ** son claramente la nación más poblada entre los países de destino con una población de más de ** 300 millones **. Todos los demás países de la lista tienen una población ** inferior a 100 millones. **

A continuación, vamos a crear un gráfico de barras de la distancia del país desde el país de origen (Estados Unidos de América). Luego, construiremos un diagrama de dispersión entre la frecuencia de reserva en un país y la distancia del país. Mi intuición inicial es que cuanto más lejos esté un país, menos probabilidades habrá de que la gente visite ese país en particular. Debe haber al menos una pequeña correlación megativa entre estas dos cantidades.

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(10,7))
sns.barplot(x='country_destination', y='distance_km', data=df_con)

In [ ]:
country_popularity = df_train[(df_train['country_destination'] != 'NDF') & (df_train['country_destination'] != 'other')]['country_destination'].value_counts()

In [ ]:
country_distance = pd.Series(df_con['distance_km'])
country_distance.index = df_con['country_destination']

In [ ]:
language_distance = pd.Series(df_con['language_levenshtein_distance'])
language_distance.index = df_con['country_destination']

In [ ]:
country_area = pd.Series(df_con['destination_km2'])
country_area.index = df_con['country_destination']

In [ ]:
df_dp = pd.concat([country_popularity, country_distance, language_distance, country_area], axis=1)
df_dp.columns = ['count', 'distance_km', 'language', 'area']
sns.jointplot(x='count', y='distance_km', data=df_dp)

Hay una ** fuerte correlación negativa de -0.65. ** La gente prefiere abrumadoramente reservar en los Estados Unidos que cualquier otro país del mundo. Podemos atribuir esta preferencia a la distancia. Verifiquemos si esto también es válido para los países internacionales.

In [ ]:
sns.jointplot(x='count', y='distance_km', data=df_dp.drop('US'))

La correlación, con Estados Unidos fuera de la ecuación, ** es mucho menor en -0.18. ** Sin embargo, todavía hay una correlación negativa entre las dos cantidades.

A continuación, verifiquemos si el idioma que se habla en un país en particular influye en el número de reservas en ese país. Para calcular esta métrica, utilizaremos el ** Idioma Levenshtein Distance ** del idioma oficial de cada país desde el inglés. Estos datos ya están disponibles para nosotros por Airbnb. Una vez más, al igual que la distancia física, mi conjetura es que a menor distancia de levenshtein, mayor es el número de reservas en ese país.

In [ ]:
sns.jointplot(x='count', y='language', data=df_dp)

Parece que hay una ** correlación media negativa de -0.38 ** entre las dos cantidades. Esto sugiere que las personas prefieren los países que hablan inglés. ¿Pero es ese el caso realmente? Eliminemos una vez más Estados Unidos, el país de origen y recalcularemos nuestro coeficiente de correlación.

In [ ]:
sns.jointplot(x='count', y='language', data=df_dp.drop('US'))

El resultado de esta visualización es extremadamente interesante.

* Vemos que cuando se elimina a Estados Unidos de la lista de países, en realidad existe ** una correlación positiva ** entre la distancia de idioma y la frecuencia de reserva.
* A primera vista, esto puede sugerir que las personas prefieren países con diferentes idiomas (y, por lo tanto, cultura) cuando viajan al extranjero.
* Otra forma de verlo sería que ** Europa es un destino de viaje muy popular para los estadounidenses ** y tienden a preferirlo a otros países de habla inglesa como Canadá y Australia. Entonces, esto puede no tener nada que ver con la diferencia de idioma, ya que puede tener que ver con la preferencia de destino.

Finalmente, en esta sección, verifiquemos si el tamaño del país tiene algo que ver con la preferencia de reserva. Al igual que en los dos casos anteriores, tenemos datos sobre el tamaño de cada país.

In [ ]:
sns.jointplot(x='count', y='area', data=df_dp)

In [ ]:
sns.jointplot(x='count', y='area', data=df_dp.drop('US'))

Los resultados con el área se asemejan mucho a lo que hemos obtenido de nuestras pruebas de distancia de idioma. En general, existe una correlación positiva entre el tamaño del país y la preferencia de reserva. Esto se debe principalmente a que Estados Unidos es un país grande (el segundo más grande en esta lista).

Sin embargo, cuando se eliminan los EE. UU., Los datos ** muestran una correlación negativa. ** Nuevamente, como en el caso del idioma, esto puede no tener nada que ver con el tamaño del país sino con la preferencia de destino. Los países europeos son generalmente pequeños en tamaño pero extremadamente populares entre los turistas estadounidenses.

### Estadísticas de Sesión

En esta sección, trataremos de visualizar algunas de las estadísticas de sesión que se nos dieron. Primero, como ya lo hemos hecho antes, representemos la distribución de la cantidad de tiempo que los usuarios dedicaron a cada sesión.


In [ ]:
sns.distplot(df_ses[df_ses['secs_elapsed'].notnull()]['secs_elapsed'])

Parece que casi todas las sesiones duraron menos de 5000 segundos. Echemos un vistazo más profundo en estas sesiones.

In [ ]:
sns.distplot(df_ses[(df_ses['secs_elapsed'].notnull()) & (df_ses['secs_elapsed'] < 5000)]['secs_elapsed'])

Podemos ver que la mayoría del número de sesiones de más de 1000 segundos disminuye casi exponencialmente. Es justo suponer que la mayoría de las sesiones duraron menos de 1000 segundos. Para verificar esto, veamos su participación en el número total de sesiones.

In [ ]:
len(df_ses[df_ses['secs_elapsed'] < 1000])/len(df_ses[df_ses['secs_elapsed'].notnull()])

** Casi el 47% de todas las sesiones duraron menos de 1000 segundos. ** Esto sugiere fuertemente una distribución exponencial decreciente de los segundos transcurridos en cada sesión. En otras palabras, a medida que aumenta la cantidad de segundos, la cantidad de instancias de sesiones que requieren ese tiempo disminuye exponencialmente.

Finalmente, en esta sección, construiremos un diagrama de barras de todos los dispositivos utilizados por varios usuarios para interactuar con el sitio web y la aplicación de Airbnb.

In [ ]:
plt.figure(figsize=(12,7))
sns.countplot(y='device_type', data=df_ses)

El ** Mac Desktop ** es el dispositivo más popular entre los usuarios de Airbnb, seguido por el ** Windows Desktop. ** Un dato interesante es que ** los productos de Apple son extremadamente populares entre los usuarios de Airbnb. ** El iPhone, el iPad y el Todos los Mac aparecen en la lista de los 5 dispositivos más populares.

### Estadísticas de usuarios de entrenamiento

En esta sección, intentaremos obtener información sobre nuestro conjunto de datos principal: el conjunto de datos de capacitación. Estas ideas se aplicarán directamente en los pasos posteriores de la ingeniería de características y el aprendizaje automático.

El primer paso es trazar la frecuencia de cada clase.

In [ ]:
plt.figure(figsize=(10,5))
country_share = df_train['country_destination'].value_counts() / df_train.shape[0] * 100
country_share.plot(kind='bar',color='#FD5C64', rot=0)
plt.xlabel('Destination Country')
plt.ylabel('Percentage')
sns.despine()

Como se puede ver arriba, cerca del ** 60% de los usuarios nunca han reservado un Airbnb. ** Entre los usuarios que lo han hecho, han elegido abrumadoramente ** Estados Unidos como su primer destino. ** Al entrenar nuestro modelo de aprendizaje automático, Nos interesa separar a los bookers de los no bookers. La clasificación subsiguiente entre los bookers daría una gran precisión, ya que podríamos usar el desequilibrio de clases a nuestro favor.

Como se puede ver arriba, cerca del ** 60% de los usuarios nunca han reservado un Airbnb. ** Entre los usuarios que lo han hecho, han elegido abrumadoramente ** Estados Unidos como su primer destino. ** Al entrenar nuestro modelo de aprendizaje automático, Nos interesa separar a los bookers de los no bookers. La clasificación subsiguiente entre los bookers daría una gran precisión, ya que podríamos usar el desequilibrio de clases a nuestro favor.

In [ ]:
classes = ['NDF','US','other','FR','IT','GB','ES','CA','DE','NL','AU','PT']

#### La visualización de barras apiladas

El gráfico de barras apiladas será la principal herramienta de visualización en la EDA. La idea se tomó de la excelente libreta de notas de kevinwu06 sobre el análisis exploratorio de Airbnb.

El gráfico de barras apiladas nos proporciona una colección de barras de colores proporcionales donde cada color representa una clase (o el destino). Como hay más de 10 clases, elegí renunciar a la leyenda. Los colores que nos interesan son ** azul oscuro (barra inferior) que corresponde a la clase NDF ** y ** azul (segunda barra inferior) que corresponde a la clase US **. Como ya hemos visto, estas dos clases forman la mayoría y, por lo tanto, ocuparán la mayor parte del área de cada barra. Cada barra ha sido normalizada con respecto a su tamaño.

Lo que intentaremos deducir es la proporción de la barra que pertenece a la clase de no reserva (azul oscuro). Si es grande, significa que es poco probable que un segmento en particular reserve un Airbnb. Si son pequeños, significa que están más inclinados a hacerlo.

In [ ]:
def stacked_bar(feature):
    ctab = pd.crosstab([df_train[feature].fillna('Unknown')], df_train.country_destination, dropna=False).apply(lambda x: x/x.sum(), axis=1)
    ctab[classes].plot(kind='bar', stacked=True, colormap='terrain', legend=False)

#### Género

In [ ]:
sns.countplot(df_train['gender'].fillna('Unknown'))

In [ ]:
stacked_bar('gender')

Vemos que los usuarios cuyo género es desconocido forman la mayoría. De los usuarios cuyo sexo se conoce, ** hay más mujeres que hombres. ** Esto puede sugerir dos cosas:
1. Hay más usuarios femeninos de Airbnb que hombres
2. Las mujeres tienen más probabilidades de revelar su género que los hombres.

Un punto muy interesante a destacar es que ** las personas que no han marcado su género tienen menos probabilidades de reservar un Airbnb. ** Además, las personas que se han marcado a sí mismas como ** 'otras' son más probables ** que cualquier otro grupo para hacer una reserva. Esto podría no ser una idea muy potente ya que el otro grupo es una proporción insignificante de la población.

#### Edad

Estudiemos ahora las edades de los usuarios y tratemos de obtener información sobre ello. Como vimos en una sección anterior, no tenemos los datos de edad de una parte considerable de los usuarios. Intentemos trazar la distribución de los que hacemos.

In [ ]:
sns.distplot(df_train['age'].dropna())

¿Es más probable que algunos grupos de edad reserven en un destino en particular? Para responder a esta pregunta, usaremos el diagrama de caja y observaremos la distribución de edades de los viajeros a un país en particular.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(15, 8))
sns.boxplot(x='country_destination', y='age', data=df_train, palette="muted", ax =ax)
ax.set_ylim([10, 75])

Desde la gráfica de boxplot, encontramos que la distribución es más o menos la misma para cada país. ** Gran Bretaña ** tiene la edad media más alta de los viajeros y ** España ** es más popular entre los viajeros más jóvenes.

In [ ]:
def set_age_group(x):
    if x < 40:
        return 'Young'
    elif x >=40 and x < 60:
        return 'Middle'
    elif x >= 60 and x <= 125:
        return 'Old'
    else:
        return 'Unknown'

In [ ]:
df_train['age_group'] = df_train['age'].apply(set_age_group)

In [ ]:
stacked_bar('age_group')

Definimos una nueva característica ** age_group ** para agrupar a nuestros usuarios en 4 grupos de edad:
* Las personas menores de 40 años son etiquetadas como jóvenes.
* Las personas con edades comprendidas entre 40 y 60 años son etiquetadas como de mediana edad.
* Las personas mayores de 60 años son etiquetadas como viejas.
* Las personas con edades desconocidas son etiquetadas como desconocidas.

Descubrimos que las personas que no han revelado su edad ** tienen menos probabilidades de reservar un Airbnb. ** De los usuarios cuya edad conocemos, ** las personas de mediana edad tienen más probabilidades de reservar un Airbnb. ** Aunque debe ser señaló que no hay una diferencia muy significativa entre los tres grupos.

#### Método de registro

En una sección anterior, ya establecimos que ** basic ** y ** Facebook ** eran los métodos de registro más populares. ** Google ** rara vez se usaba.

In [ ]:
stacked_bar('signup_method')

Descubrimos que las personas que usan el método de registro ** básico ** tienen más probabilidades de reservar un Airbnb, mientras que las personas que usan ** google ** tienen menos probabilidades de hacer una reserva.

#### Idioma

In [ ]:
stacked_bar('language')

Vemos que las personas que hablan ** húngaro ** y ** indonesio ** casi no hacen reservas. Las personas que hablaron ** finlandés ** hicieron la mayoría de las reservas entre todos los idiomas. Sin embargo, dado que los usuarios son estadounidenses, podemos asumir con seguridad que los usuarios que no son ingleses constituyen una minoria.

#### Canales de afiliados y proveedores

In [ ]:
stacked_bar('affiliate_channel')

In [ ]:
plt.figure(figsize=(8,4))
sns.countplot(df_train['affiliate_channel'])

In [ ]:
stacked_bar('affiliate_provider')

In [ ]:
plt.figure(figsize=(18,4))
sns.countplot(df_train['affiliate_provider'])

In [ ]:
stacked_bar('first_affiliate_tracked')

Algunas observaciones de las visualizaciones anteriores:
* El ** Canal Directo ** tiene la mayor cantidad de conversiones a las reservas, mientras que el ** Canal de Contenido ** tiene la menor cantidad.
* ** Direct ** y ** Google ** son los proveedores afiliados más populares.
* ** Wayn ** tiene el menor porcentaje de conversiones mientras que ** Daum ** tiene el mayor porcentaje. Sin embargo, debemos tomar esto con una pizca de sal, ya que puede ser que el número de puntos de muestra de estas categorías sea extremadamente pequeño (como lo sugiere la gráfica de conteo).
* Aparte de lo anterior, ** Google ** y ** Craigslist ** tienen un buen porcentaje de conversiones.
* Las personas con ** Marketing ** afiliados eran más propensas a reservar. Las personas cuyo primer afiliado fue registrado como ** Operaciones locales ** o ** Desconocido ** fueron las menos probables.

#### Flujo de Inscripción

In [ ]:
stacked_bar('signup_flow')

El gráfico de barras apiladas del flujo de inscripción genera observaciones muy interesantes: ** las personas con flujo de registro 3 ** tienen más probabilidades de reservar un Airbnb. Por el contrario, las personas con flujos de registro ** 4 y 5 tienen menos probabilidades de convertirse. **

#### Tipo de dispositivo, navegador y aplicación utilizados

In [ ]:
stacked_bar('signup_app')

In [ ]:
stacked_bar('first_device_type')

Como hay demasiados navegadores, lo ignoraremos por el momento e intentaremos reducir las categorías en un paso posterior. Por ahora, podemos hacer las siguientes observaciones sobre los dispositivos de usuario:
* Los usuarios que usan la aplicación web ** tienen más probabilidades de reservar un Airbnb, mientras que ** los usuarios de Android ** tienen menos probabilidades de hacerlo.
* ** Las personas con un teléfono Android o cuyos dispositivos eran desconocidos ** compraron menos Airbnbs. La gente en computadoras de escritorio (Mac o de otro tipo) compró más.

Esto sugiere fuertemente que los usuarios en su escritorio tendrán más probabilidades de reservar un Airbnb y los Usuarios de Apple son más propensos a comprar en el sitio web, mientras que los Usuarios de Android son los menos.

#### Fechas

Revisemos primero las tendencias acerca de cuándo se crearon las primeras cuentas.

In [ ]:
df_train['date_account_created'] = pd.to_datetime(df_train['date_account_created'])

In [ ]:
sns.set_style("whitegrid", {'axes.edgecolor': '0'})
sns.set_context("poster", font_scale=1.1)
plt.figure(figsize=(12,6))

df_train[df_train['country_destination'] != 'NDF']['date_account_created'].value_counts().plot(kind='line', linewidth=1, color='green')
df_train[df_train['country_destination'] == 'NDF']['date_account_created'].value_counts().plot(kind='line', linewidth=1, color='red')

El número de usuarios que no reservan ha aumentado más que el número de usuarios que reservan a medida que pasa el tiempo. Esto se puede atribuir al hecho de que más personas están utilizando la plataforma solo para la exploración. Otra razón podría ser que dado que el usuario estuvo en la plataforma por un tiempo más prolongado, era más probable que él / ella siguiera adelante y reservara un espacio. Esto implica directamente que ** cuanto antes el usuario había creado una cuenta en Airbnb, era más probable que hiciera una reserva. **

Con todo esto en mente, no podemos continuar con el Paso de ingeniería de características. Las características que aún no hemos explorado en esta sección se explorarán en la siguiente.

## Ingeniería de características

Ahora, con todas estas visualizaciones e ideas en nuestro haber, procedamos a desarrollar algunas características nuevas que nos ayudarán a obtener mejores predicciones cuando finalmente capacitemos nuestro modelo de aprendizaje automático. Primero, me gustaría extraer características de nuestro conjunto de datos de sesiones. Además, dado que no hemos hecho demasiada visualización de nuestras sesiones, este sería un buen lugar para hacerlo.

In [ ]:
df_ses.head(2)

> ### Características de la sesión

La primera característica que quiero diseñar es la cantidad de dispositivos a los que un usuario accede a la aplicación. ** Mi corazonada es que si el usuario usa muchos dispositivos, implica que viaja muy a menudo y, por lo tanto, es probable que reserve un Airbnb. ** Una hipótesis adicional es que si la persona viaja mucho, s / probablemente debe ser un viajero de negocios y, por lo tanto, es probable que esté dispuesto a reservar Airbnbs dentro de los Estados Unidos.

La segunda característica que quiero es el ** número total de segundos que el usuario ha gastado en las sesiones de Airbnb. ** Será interesante ver cómo esto se correlaciona con nuestras clases.

La tercera característica es ** segundos promedio por sesión. ** Nuevamente, no tengo mucha intuición detrás de este paso, pero tengo la sensación de que esta característica puede ser útil.

La cuarta característica es ** el número total de sesiones. ** Al igual que con los segundos totales, esto podría indicar un mayor interés. Además, también construiremos una característica de ** número de tipos de sesión ** para cuantificar aún más el interés y el compromiso del usuario.

La última característica que quiero ver es ** número de sesiones cortas. ** Estas son sesiones de menos de 300 segundos de duración. Construiremos un ** número de sesiones largas ** al restarlo del número total de sesiones.

In [ ]:
def session_features(df):
    df['total_seconds'] = df['id'].apply(lambda x: total_seconds[x] if x in total_seconds else 0)
    df['average_seconds'] = df['id'].apply(lambda x: average_seconds[x] if x in average_seconds else 0)
    df['total_sessions'] = df['id'].apply(lambda x: total_sessions[x] if x in total_sessions else 0)
    df['distinct_sessions'] = df['id'].apply(lambda x: distinct_sessions[x] if x in distinct_sessions else 0)
    df['num_short_sessions'] = df['id'].apply(lambda x: num_short_sessions[x] if x in num_short_sessions else 0)
    df['num_long_sessions'] = df['id'].apply(lambda x: num_long_sessions[x] if x in num_long_sessions else 0)
    df['num_devices'] = df['id'].apply(lambda x: num_devices[x] if x in num_devices else 0)
    return df

In [ ]:
def browsers(df):
    df['first_browser'] = df['first_browser'].apply(lambda x: "Mobile_Safari" if x == "Mobile Safari" else x)
    major_browsers = ['Chrome', 'Safari', 'Firefox', 'IE', 'Mobile_Safari']
    df['first_browser'] = df['first_browser'].apply(lambda x: 'Other' if x not in major_browsers else x)
    return df

In [ ]:
def classify_device(x):
    if x.find('Desktop') != -1:
        return 'Desktop'
    elif x.find('Tablet') != -1 or x.find('iPad') != -1:
        return 'Tablet'
    elif x.find('Phone') != -1:
        return 'Phone'
    else:
        return 'Unknown'

In [ ]:
def devices(df):
    df['first_device_type'] = df['first_device_type'].apply(classify_device)
    return df

In [ ]:
def affiliate_tracked(df):
    df['first_affiliate_tracked'] = df['first_affiliate_tracked'].fillna('Unknown')
    df['first_affiliate_tracked'] = df['first_affiliate_tracked'].apply(lambda x: 'Other' if x != 'Unknown' and x != 'untracked' else x)
    return df

In [ ]:
def affiliate_provider(df):
    df['affiliate_provider'] = df['affiliate_provider'].apply(lambda x: 'rest' if x not in ['direct', 'google', 'other'] else x)
    return df

In [ ]:
def affiliate_channel(df):
    df['affiliate_channel'] = df['affiliate_channel'].apply(lambda x: 'other' if x  not in ['direct', 'content'] else x)
    return df

In [ ]:
def languages(df):
    df['language'] = df['language'].apply(lambda x: 'foreign' if x != 'en' else x)
    return df

In [ ]:
def first_booking(df):
    df = df.drop('date_first_booking', axis=1)
    return df

In [ ]:
def account_created(df):
    df = df.drop('date_account_created', axis=1)
    return df

In [ ]:
def feature_engineering(df):
    df = session_features(df)
    df = df.drop('age', axis=1)
    df = browsers(df)
    df =devices(df)
    df =affiliate_tracked(df)
    df = affiliate_provider(df)
    df = affiliate_channel(df)
    df = languages(df)
    df['is_3'] = df['signup_flow'].apply(lambda x: 1 if x==3 else 0)
    df = first_booking(df)
    df = df.drop('timestamp_first_active', axis=1)
    df = account_created(df)
    df = df.set_index('id')
    df = pd.get_dummies(df, prefix='is')
    return df

In [ ]:
total_seconds = df_ses.groupby('user_id')['secs_elapsed'].sum()

In [ ]:
average_seconds = df_ses.groupby('user_id')['secs_elapsed'].mean()

In [ ]:
total_sessions = df_ses.groupby('user_id')['action'].count()

In [ ]:
distinct_sessions = df_ses.groupby('user_id')['action'].nunique()

In [ ]:
num_short_sessions = df_ses[df_ses['secs_elapsed'] <= 300].groupby('user_id')['action'].count()
num_long_sessions = df_ses[df_ses['secs_elapsed'] >= 2000].groupby('user_id')['action'].count()

In [ ]:
num_devices = df_ses.groupby('user_id')['device_type'].nunique()

In [ ]:
df_train = session_features(df_train)

Since we already have our age groups, we can drop the age feature.

In [ ]:
df_train = df_train.drop('age', axis=1)

Hay más de 50 categorías de navegadores, aunque la mayoría utiliza solo 5. Por lo tanto, agruparemos todos los exploradores menores en la categoría 'Otros'.

Luego, de nuestra visualización anterior, vimos que los usuarios de computadoras de escritorio tenían más probabilidades de reservar Airbnbs, seguidos por los usuarios de Tabletas, usuarios de teléfonos y finalmente personas cuyos dispositivos eran desconocidos. Vamos a reducir first_device_type en estas categorías.

In [ ]:
df_train = browsers(df_train)

In [ ]:
df_train = devices(df_train)

Hagamos lo mismo con los afiliados.

In [ ]:
df_train = affiliate_tracked(df_train)

In [ ]:
df_train = affiliate_provider(df_train)

In [ ]:
df_train = affiliate_channel(df_train)

El idioma utilizado es mayoritariamente el inglés. Por lo tanto, tiene sentido apilar a los demás.

In [ ]:
df_train = languages(df_train)

En nuestra sección de EDA, notamos que las personas con flujo de registro 3 tuvieron un número desproporcionado de conversiones. Por lo tanto, definiremos una característica adicional que identifique a estos usuarios. También revertiremos nuestra fatiga de género al rellenar los NaN con Desconocido ya que deducimos que esto llevará a una mejora en la predicción.

In [ ]:
df_train['is_3'] = df_train['signup_flow'].apply(lambda x: 1 if x==3 else 0)
df_train['gender'] = df_train['gender'].fillna('Unknown')

Vamos a soltar las características de first_bookings y timsestamp. Esto se debe a que no agregan ningún valor cuando finalmente prueban el modelo con el conjunto de pruebas. Todos nuestros usuarios de prueba tienen NaN como primera reserva y la marca de tiempo no tiene importancia.

In [ ]:
df_train = first_booking(df_train)

In [ ]:
df_train = df_train.drop('timestamp_first_active', axis=1)

Finalmente, eliminaremos la función de cuentas creadas, ya que nuestros conjuntos de entrenamiento y prueba se separaron a mediados de 2014. Como resultado, todos los usuarios de prueba se registraron solo en 2014.

In [ ]:
df_train = account_created(df_train)

In [ ]:
df_train = df_train.set_index('id')

Nuestro conjunto de datos ahora está en posición de tener una codificación activa realizada en él. Separemos ahora nuestros datos de X y de y.

In [ ]:
class_dict = {
    'NDF': 0,
    'US': 1,
    'other': 2,
    'FR': 3,
    'CA': 4,
    'GB': 5,
    'ES': 6,
    'IT': 7,
    'PT': 8,
    'NL': 9,
    'DE': 10,
    'AU': 11
}

In [ ]:
X, y = df_train.drop('country_destination', axis=1), df_train['country_destination'].apply(lambda x: class_dict[x])

In [ ]:
X = pd.get_dummies(X, prefix='is')

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.75, stratify=y)

## Aprendizaje automático


El siguiente paso es construir un clasificador para entrenar nuestros datos y luego probar su rendimiento con los datos de prueba. Con toda la ingeniería de características ya realizada en el paso anterior, la aplicación del aprendizaje automático debe ser bastante concisa.

### Selección de modelo

Sin embargo, debemos crear un clasificador que tenga el mejor rendimiento, dadas las características. En tales competiciones, Ensemble Methods dan los mejores resultados. Entrenaremos a nuestro modelo utilizando tres clasificadores: ** Incremento de gradiente **, ** Regresión logística ** y ** Bosque aleatorio ** y elegiremos el que tenga la mejor precisión.

In [ ]:
#classifiers = [RandomForestClassifier(verbose=1), LogisticRegression(verbose=1), GradientBoostingClassifier(verbose=True)]

#for classifier in classifiers:
#    classifier.fit(train_X, train_y)
#    print("Score: " + str(classifier.score(test_X, test_y)))

Los siguientes resultados fueron obtenidos:
* ** Regresión logística ** es el clasificador menos preciso con una precisión registrada inferior al 60%.
* En promedio, el ** Clasificador de aumento de gradiente ** obtuvo el mejor rendimiento con una precisión cercana al 65%.

Por supuesto, hay otros clasificadores que pueden probarse en este paso, pero me detendré aquí y elegiré ** el Clasificador de refuerzo de gradiente ** como mi modelo de elección.

Este clasificador tiene una precisión de ** aproximadamente el 65% **. Intentemos y mejoremos esto utilizando el ajuste de hiperparámetro usando la validación cruzada de búsqueda de cuadrícula. Afortunadamente, como siempre, scikit-learn tiene las herramientas disponibles para nosotros que reducen la cantidad de código al mínimo.

### Ajuste del hiperparámetro usando Grid Search Cross Validation

In [ ]:
parameters = {
    'n_estimators': [100,200],
    'max_features': ['auto', 'log2'],
    'max_depth': [3,5]
}

In [ ]:
#clf = GridSearchCV(GradientBoostingClassifier(), parameters, verbose=100)
#clf.fit(train_X, train_y)

In [ ]:
#clf.best_params_

Los mejores parámetros obtenidos por Grid Search se pueden ver arriba. Cambiaremos la tasa de aprendizaje a 200 para obtener mejores resultados.

### Pruebas y envío final

Ahora que tenemos un clasificador que ha sido optimizado para obtener mejores resultados, podemos proceder a hacer predicciones para nuestros usuarios de prueba y crear el archivo de envío final.

In [ ]:
df_test = pd.read_csv('../input/test_users.csv')
df_test['gender'] = df_test['gender'].replace('-unknown-', 'Unknown')
df_test['age_group'] = df_test['age'].apply(set_age_group)
df_test.head()

In [ ]:
#df_test = feature_engineering(df_test)
#df_test = df_test.drop('is_weibo', axis=1)

In [ ]:
#df_test.columns

In [ ]:
#X.columns

In [ ]:
#pred_prob = clf.predict_proba(df_test)

In [ ]:
#pred_prob = pd.DataFrame(pred_prob, index=df_test.index)
#pred_prob.head()

In [ ]:
#inv_classes = {v: k for k, v in class_dict.items()}
#inv_classes

In [ ]:
def get_top(s):
    indexes = [i for i in range(0,12)]
    lst = list(zip(indexes, s))
    top_five = sorted(lst, key=lambda x: x[1], reverse=True)[:5]
    top_five = [inv_classes[i[0]] for i in top_five]
    return str(top_five)

In [ ]:
#pred_prob['get_top'] = pred_prob.apply(get_top, axis=1)
#pred_prob.head()

In [ ]:
#import ast
#pred_prob['get_top'] = pred_prob['get_top'].apply(lambda x: ast.literal_eval(x))

In [ ]:
#s = pred_prob.apply(lambda x: pd.Series(x['get_top']),axis=1).stack().reset_index(level=1, drop=True)
#s.name = 'country'

In [ ]:
#submission = pred_prob.drop([i for i in range(0,12)] + ['get_top'], axis=1).join(s)
#submission.head()

In [ ]:
#submission.to_csv('submission.csv')

## Recomendaciones de negocios

En función del análisis exploratorio de datos y el modelado predictivo realizados, podemos hacer las siguientes sugerencias a nuestro cliente (Airbnb):

* La mayoría de los usuarios de Airbnb están en dispositivos Apple. Además, los usuarios de Android son la minoría y no reservan tan a menudo. ** Es lógico para los negocios invertir más recursos en mejorar la experiencia de usuario de las aplicaciones iOS. ** Por otro lado, Airbnb también podría investigar el menor índice de conversión de los usuarios de Android e intentar deducir cierta correlación con ciertas peculiaridades del Aplicación de Android Airbnb.
* Los usuarios de escritorio reservan con mucha más frecuencia que otros usuarios de dispositivos. ** Esto podría significar que la gente use dispositivos más pequeños para navegar y explorar, y escritorios para hacer la reserva real. ** La experiencia del usuario para las aplicaciones en diferentes dispositivos debe ajustarse en consecuencia para lograr los mejores resultados.
* La mayoría de los usuarios de Airbnb tienden a reservar un Airbnb dentro del país. ** Por lo tanto, tiene más sentido ofrecer a los usuarios más recomendaciones que sean endémicas del país. ** Los usuarios estadounidenses deben obtener más recomendaciones de Experiencia y Hospedaje dentro de los EE. UU. Este no es el caso con el sitio actual de Airbnb, donde la mayoría de las recomendaciones son lugares exóticos en lugares lejanos.
* Las personas que no han completado su información, como Edad y Género, tienen menos probabilidades de reservar Airbnbs. Esto probablemente se deba al hecho de que solo se encuentran en la etapa de exploración. ** El máximo esfuerzo de marketing debe estar dirigido a estas personas y se les debe hacer las ofertas más atractivas pero asequibles para aumentar su tasa de conversión. **

## Conclusión

Este cuaderno demostró todos los pasos principales que tienen lugar en el análisis de datos de desempeño y el modelado predictivo en un problema típico de la ciencia de datos. Los datos fueron manipulados y limpiados. Esto fue seguido por algunas estadísticas inferenciales para deducir las relaciones entre las características. Se realizó una EDA extensa para obtener información sobre los datos y se usaron para extraer e diseñar nuevas características.

Finalmente, se consideraron varios clasificadores y se seleccionó el ** Gradient Boosting Classfier ** y se ajustaron sus parámetros. Este clasificador se utilizó para predecir el destino de nuestros usuarios de prueba y se creó un archivo de envío en el formato sugerido por Airbnb.

El puntaje oficial de Kaggle obtenido fue ** 0,86527. ** Este puntaje puede, por supuesto, mejorarse a través de un ajuste más hiperparamétrico y una ingeniería de funciones más avanzada, pero la mejora es extremadamente mínima. Por lo tanto voy a concluir este cuaderno aquí. ¡Gracias!